# Imports

In [ ]:
from jupyter_dash import JupyterDash

import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import numpy as np
import plotly.express as px

from wordcloud import WordCloud
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
from PIL import Image

from PIL.ExifTags import GPSTAGS, TAGS

import calendar

import  json
import os

import random

JupyterDash.infer_jupyter_proxy_config()

# Daten
Die Daten werden in diesem Notebook zufällig generiert, damit die Funktionalität dieses Notebooks gewährleistet werden kann. 

In [2]:
categories = [
    'weight', 
    'bodyFat (kg)', 
    'time in bed (min)',
    'sleepTime',
    'min-hr',
    'steps',
    'totalMinActive', 
    'minMVPA',
    'times run',
    'runDuration (min)',
    'runDistance (km)',
    'runHeartRate',      
    'strengthTraining', 
    'pace', 
    'net steps',
]
abbreviations = {
    'Business Day': 'B',
    'Calendar Day': 'D',
    'Week': 'W',
    'Month': 'M',
    'Quarter': 'Q',
    'Year': 'Y'
}
time_spans = list(abbreviations.keys())
# Die Funktionen zum Aggregieren können hier nachgelesen werden:
# https://numpy.org/doc/stable/reference/routines.math.html
aggr_options = ['mean', 'sum', 'min', 'max', 'prod', 'std', 'var', 'median']
aggr_dict = {
    # möglich: mean, sum, nansum, min, max, prod, std, var, median, percentile
    'weight': np.mean,
    'bodyFat (kg)': np.mean,
    'time in bed (min)': np.mean,
    'sleepTime': np.mean,
    'min-hr': np.mean,
    'steps': np.mean,
    'totalMinActive': np.mean, 
    'minMVPA': np.mean,
    'times run': np.sum,
    'runDuration (min)': np.sum,
    'runDistance (km)': np.nansum,
    'runHeartRate': np.mean,     
    'strengthTraining': np.mean,
    'pace': np.mean,
    'net steps': np.mean,
}
information_dict = {category:'None' for category in categories}
factors = {
    'weight': random.randrange(50, 100), 
    'bodyFat (kg)': random.randrange(3, 30),  
    'time in bed (min)': random.randrange(400, 800),
    'sleepTime': random.randrange(300, 1500),
    'min-hr': random.randrange(70, 150),
    'steps': random.randrange(7000, 25000),
    'totalMinActive': random.randrange(10, 1000),
    'minMVPA': random.randrange(100, 600),
    'times run': random.randrange(1, 5),
    'runDuration (min)': random.randrange(10, 200),
    'runDistance (km)': random.randrange(1000, 20000),
    'runHeartRate': random.randrange(100, 190),
    'strengthTraining': random.randrange(1, 3),
    'pace': random.random()/10.0,
    'net steps': random.randrange(3000, 17000), 
}

first_date = datetime.date(2010, 1, 1)
second_date = datetime.date(2020, 12, 31)
index = pd.date_range(start=first_date, end=second_date)

data = {}
for category in categories:
    y = 0
    temp_array = []
    for i in range(len(index)):
        value = y + factors[category]
        temp_array.append(value if random.random() > 0.1 else None)
        y += np.random.normal(scale=1)
    data.update({category: temp_array})
data = pd.DataFrame(data=data, index=index)
data['Date'] = [temp_string.strftime("%d-%m-%Y") for temp_string in data.index]
data['pace'] = data['pace'] * 1000

In [3]:
words = ['Abbey', 'Academic conference', 'Adult', 'Adventure',
   'Advertising', 'Aerial photography', 'Afterglow', 'Agriculture',
   'Air travel', 'Aircraft', 'Airline', 'Airliner', 'Airplane',
   'Alcoholic beverage', 'Alley', 'Alps', 'Altar', 'Ancient history',
   'Ancient roman architecture', 'Animal', 'Arch', 'Arch bridge',
   'Archaeological site', 'Architecture', 'Arecales', 'Arena', 'Arm',
   'Art', 'Asphalt', 'Athlete', 'Athletics', 'Audience', 'Auditorium',
   'Automotive exterior', 'Autumn', 'Aviation', 'Baby', 'Backyard',
   'Badlands', 'Baked goods', 'Baking', 'Bar', 'Basilica',
   'Battleship', 'Bay', 'Bazaar', 'Beach', 'Beak', 'Beard', 'Bed',
   'Beer', 'Bell tower', 'Bike', 'Biome', 'Bird', "Bird's-eye view",
   'Black', 'Black-and-white', 'Blossom', 'Blue', 'Boardwalk', 'Boat',
   'Boating', 'Body of water', 'Boeing 777', 'Boulder', 'Boy',
   'Branch', 'Brand', 'Breakfast', 'Breakwater', 'Brick', 'Brickwork',
   'Bridge', 'Building', 'Bumper', 'Byzantine architecture',
   'Cabinetry', 'Calligraphy', 'Canal', 'Cape', 'Car', 'Carnival',
   'Carving', 'Castle', 'Cathedral', 'Cave', 'Caving', 'Ceiling',
   'Cemetery', 'Channel', 'Chapel', 'Child', 'Chinese food',
   'Christmas', 'Christmas decoration', 'Christmas lights', 'Church',
   'Château', 'Circle', 'City', 'City car', 'Cityscape', 'Cliff',
   'Close-up', 'Clothing', 'Cloud', 'Coast', 'Cobblestone', 'Column',
   'Commercial vehicle', 'Condominium', 'Convenience store',
   'Convention', 'Cottage', 'Courtyard', 'Cove', 'Crop', 'Crowd',
   'Cuisine', 'Cup', 'Cycle sport', 'Cycling', 'Darkness', 'Dawn',
   'Daylighting', 'Desert', 'Design', 'Dessert', 'Destroyer',
   'Diagram', 'Dish', 'Distilled beverage', 'Dock', 'Document', 'Dog',
   'Dome', 'Downtown', 'Drawing', 'Drink', 'Driving', 'Duathlon',
   'Duck', 'Ducks, geese and swans', 'Dusk', 'Electricity',
   'Emergency service', 'Emergency vehicle', 'Endurance sports',
   'Engine', 'Estate', 'Evening', 'Event', 'Extreme sport', 'Eye',
   'Eyewear', 'Facade', 'Face', 'Facial hair', 'Factory',
   'Family car', 'Farm', 'Fashion accessory', 'Fell', 'Female',
   'Ferry', 'Festival', 'Field', 'Finger', 'Fire department',
   'Fireworks', 'Fjord', 'Flight', 'Floor', 'Flooring', 'Floristry',
   'Flower', 'Fog', 'Font', 'Food', 'Footwear', 'Forest', 'Formation',
   'Fortification', 'Freeway', 'Freezing', 'Frigate', 'Frost', 'Fun',
   'Furniture', 'Garden', 'Geology', 'Glass', 'Glasses',
   'Gothic architecture', 'Grass', 'Grassland', 'Grave', 'Green',
   'Grocery store', 'Guided missile destroyer', 'Gull', 'Hair',
   'Hairstyle', 'Half marathon', 'Hand', 'Harbor', 'Hardwood', 'Haze',
   'Head', 'Headstone', 'Helicopter', 'Highland', 'Highway', 'Hiking',
   'Hill', 'Hindu temple', 'Historic site', 'Home', 'Horizon',
   'House', 'Human body', 'Ice', 'Illustration', 'Individual sports',
   'Infrastructure', 'Interaction', 'Interior design', 'Iron',
   'Jet aircraft', 'Jewellery', 'Jungle', 'Kitchen', 'Label', 'Lake',
   'Land lot', 'Land vehicle', 'Landmark', 'Landscape', 'Lane',
   'Lawn', 'Leaf', 'Leg', 'Leisure', 'Library', 'Licence plate',
   'Light', 'Lighting', 'Limb', 'Line', 'Liqueur', 'Living room',
   'Loch', 'Locomotive', 'Long-distance running', 'Lunch',
   'Luxury vehicle', 'Male', 'Mammal', 'Man', 'Mansion', 'Map',
   'Marathon', 'Marina', 'Market', 'Marketplace', 'Marsh', 'Mast',
   'Meadow', 'Meal', 'Meat', 'Memorial', 'Metropolis',
   'Metropolitan area', 'Military', 'Minibus', 'Mist', 'Modern art',
   'Monastery', 'Monochrome', 'Monochrome photography', 'Monument',
   'Morning', 'Motor vehicle', 'Motorcycle', 'Mountain',
   'Mountain bike', 'Mountain biking', 'Mountain pass',
   'Mountain range', 'Mountaineering', 'Mouth', 'Muscle', 'Music',
   'Musician', 'Nail', 'Narrow-body aircraft', 'Natural environment',
   'Nature', 'Naval ship', 'Navy', 'Neighbourhood', 'Net', 'Night',
   'Noodle', 'Nose', 'Number', 'Ocean', 'Off-roading', 'Orchestra',
   'Organ', 'Outdoor recreation', 'Outdoor structure',
   'Overhead power line', 'Painting', 'Palace', 'Panorama', 'Park',
   'Passenger ship', 'Pasture', 'Pattern', 'Pedestrian', 'People',
   'Performance', 'Person', 'Pet', 'Photograph', 'Picture frame',
   'Pier', 'Place of worship', 'Plain', 'Plant', 'Plantation',
   'Plateau', 'Play', 'Playground', 'Plaza', 'Police', 'Pond', 'Port',
   'Portrait', 'Poster', 'Prairie', 'Produce', 'Property',
   'Public space', 'Quail', 'Race', 'Railroad car', 'Railway',
   'Rainforest', 'Recreation', 'Red', 'Red sky at morning',
   'Reflection', 'Reservoir', 'Residential area', 'Restaurant',
   'Retail', 'Ridge', 'River', 'Road', 'Road bicycle', 'Road cycling',
   'Road surface', 'Road trip', 'Rock', 'Rolling stock', 'Roof',
   'Room', 'Rotorcraft', 'Ruins', 'Running', 'Rural area', 'Sand',
   'Savanna', 'Screenshot', 'Sculpture', 'Sea', 'Seabird', 'Selfie',
   'Senior citizen', 'Shelf', 'Ship', 'Shore', 'Shrub', 'Sidewalk',
   'Sign', 'Signage', 'Sitting', 'Sketch', 'Skin', 'Sky', 'Skyline',
   'Skyscraper', 'Sleeve', 'Smile', 'Snack', 'Snow',
   'Soccer-specific stadium', 'Social group', 'Soil', 'Soup', 'Spire',
   'Sport utility vehicle', 'Sport venue', 'Sports',
   'Sports equipment', 'Spring', 'Sprint', 'Stadium', 'Stage',
   'Stained glass', 'Stately home', 'Statue', 'Steeple', 'Steppe',
   'Stone carving', 'Stone wall', 'Street', 'Street light',
   'Street sign', 'Suburb', 'Summit', 'Sun', 'Sunglasses', 'Sunlight',
   'Sunrise', 'Sunset', 'Supermarket', 'Swimming pool', 'Symmetry',
   'Synagogue', 'T-shirt', 'Table', 'Tarmac', 'Tarn', 'Temple',
   'Terrain', 'Text', 'Textile', 'Toddler', 'Tourism',
   'Tourist attraction', 'Tower', 'Tower block', 'Town',
   'Town square', 'Track', 'Track and field athletics',
   'Traffic sign', 'Trail', 'Train', 'Transport', 'Travel', 'Tree',
   'Truck', 'Tundra', 'Ultramarathon', 'Urban area', 'Vacation',
   'Valley', 'Van', 'Vegetable', 'Vegetation', 'Vehicle', 'Village',
   'Vision care', 'Wadi', 'Walking', 'Walkway', 'Wall', 'Warship',
   'Water', 'Water bird', 'Water feature', 'Watercraft', 'Waterfall',
   'Waterfowl', 'Waterway', 'Wave', 'Wetland', 'Wheel', 'White',
   'Wide-body aircraft', 'Wilderness', 'Wildlife', 'Wind',
   'Wind wave', 'Window', 'Wing', 'Winter', 'Wood', 'Woodland',
   'Yard', 'Zoo']

In [4]:
sample_size = 1694

years = np.arange(first_date.year, second_date.year+1)
holiday_size = 14
index = []
for year in years:
    year_range = pd.date_range(start=datetime.datetime(year, 1, 1), periods=365, freq='D')
    holiday_start = random.randrange(start=0, stop=365-holiday_size)
    for date in year_range[holiday_start:holiday_start+holiday_size]:
        index.append(np.repeat(date, len(years)))

index = np.array(index).flatten()

files = [date.strftime("%Y") + "-" + str(random.random()) + ".jpg" for date in index]
album_names = np.repeat(random.sample(words, k=len(years)), len(years)*holiday_size)
albums = [date.strftime("%Y") + " " + album_names[i] for i, date in enumerate(index)]

random_latitude = [random.randrange(start=-90, stop=90) for _ in range(len(years))]
random_latitude = np.repeat(random_latitude, len(years)*holiday_size)
random_latitude = [item + random.random() for item in random_latitude]

random_longitude = [random.randrange(start=-180, stop=180) for _ in range(len(years))]
random_longitude = np.repeat(random_longitude, len(years)*holiday_size)
random_longitude = [item + random.random() for item in random_longitude]

word_array = []
for i in range(sample_size):
    dict_size = random.randrange(start=1, stop=10, step=1)
    temp_dict = {word: str(random.random()) for word in random.sample(words, dict_size)}
    temp_dict_string = json.dumps(temp_dict)
    word_array.append(temp_dict_string)

df_pictures = pd.DataFrame(
    data={
        'file': files,
        'album': albums,
        'longitude': random_longitude,
        'latitude': random_latitude,
        'words': word_array
    }, 
    index=index
)

## Dashboard Layout

In [6]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

# Create server variable with Flask server object for use with gunicorn
server = app.server

In [7]:
#temp = []
#for category in categories:
#    category_aggr_function = category + " aggregate function:"
#    dropdown_id = category+"_selector"
#    temp.append(
#            html.Div([
#                    html.Label(category_aggr_function),
#                    dcc.Dropdown(
#                        id=dropdown_id,
#                        options=[{'label': i, 'value': i} for i in ['mean', 'sum', 'min', 'max', 'prod', 'std', 'var', 'median', 'percentile']],
#                        value= 'mean' 
#                    ),
#                ],
#                style={'padding': '10px 5px', 'width': '100%'}
#            )
#    )

#temp.append(html.Hr())
#temp.append(
#    html.Div([
#        html.Label('Categories to show in the table:'),
#        dcc.Dropdown(
#            id='categories_selector',
#            options=[{'label': category, 'value': category} for category in categories],
#            value=categories[:2],
#            multi=True
#        ),
#    ],style={'padding': '10px 5px', 'width': '100%'}
#    ),
#)

#settings_layout = html.Div(
#    [
#        html.Div([
#            html.Div(
#                temp,
#                style={'width': '49%', 'padding': '0 20', 'display':'inline-block'}
#            ),
#            html.Div([
#                html.Label('Interpolate..'),
#                dcc.Checklist(
#                    options=[{'label': category, 'value': category} for category in categories],
#                    value=categories[:2]
#                )  
#            ],style={'width': '49%', 'padding': '0 20', 'display':'inline-block', 'float':'right', 'vertical-align': 'top'}),
#            
#        ], style={'width': '100%', 'padding': '0 20'}),
#        
#        
#        dash_table.DataTable(
#            id='data_table',
#            columns=[{"name": category, "id": category} for category in categories],
#            data=data[categories].to_dict('records'),
#            page_size=20,
#        )
#    ],
#    style={'width': '100%', 'padding': '0 20'}
#)    

In [8]:
dash_layout = html.Div([
    html.Div([
        html.Div([
            html.Div([
                html.Label('Plot-Type'),
                dcc.Dropdown(
                    id='plot_type_selector',
                    options=[{'label': i, 'value': i} for i in ['Boxplot', 'Scatterplot', 'Heatmap', 'Densitymap']],
                    value='Densitymap'
                ),
            ],
            style={'padding': '10px 5px'}),
            
            html.Div([
                html.Label('First Category'),
                dcc.Dropdown(
                    id='category_one_selector',
                    options=[{'label': category, 'value': category} for category in categories],
                    value=categories[0]
                ),  
                
            ],
            style={'padding': '10px 5px'}),

            html.Div([
                html.Label('Second Category'),
                dcc.Dropdown(
                    id='category_two_selector',
                    options=[{'label': category, 'value': category} for category in categories],
                    value=categories[1]
                ),
            ], 
            style={'padding': '10px 5px'}),

            html.Div([
                html.Label('Degree Of Detail'),
                dcc.Dropdown(
                    id='detail_selector',
                    options=[{'label': span, 'value': span} for span in time_spans],
                    value=time_spans[0]
                ),
            ],
            style={'padding': '10px 5px'}),

            html.Div([
                html.Label('Year Selector'),
                dcc.RangeSlider(
                    id='year_selector',
                    min=first_date.year,
                    max=second_date.year,
                    #value=data.index.year.max(),
                    value=[2013, 2015],
                    allowCross=False,
                    marks={year: '\'{}'.format(year[-2:]) for year in np.arange(first_date.year, second_date.year+1).astype(str)},
                    step=None,
                    included=True,
                ),
            ],
            style={'padding': '20px 20px 20px 20px'}),
            
            html.Div([
                html.Label('Average\'s Selector'),
                dcc.RangeSlider(
                    id='averages_selector',
                    min=0,
                    max=365,
                    step=5,
                    value=[30, 60],
                    marks={0: '0', 30:'30', 60:'60', 100:'100', 200:'200', 365:'365'},
                    included=False
                ),
            ],
            style={'padding': '20px 20px 20px 20px'}),
            
            html.Div([
                html.Label('Amount of words for Wordclout'),
                dcc.Slider(
                    id='word_cloud_count',
                    min=1,
                    max=50,
                    step=5,
                    value=20,
                    marks={1 if i == 0 else i*10: str(1) if i == 0 else str(i*10) for i in range(0, 6)},
                    included=False
                ),
            ],
            style={'padding': '20px 20px 20px 20px'}),
            
            html.Div([
                html.Label('Min. Confidence level for words in Wordcloud'),
                dcc.Slider(
                    id='word_cloud_confidence_level',
                    min=0.0,
                    max=1.0,
                    step=0.1,
                    value=0.5,
                    marks={i/10: str(i/10) for i in range(0, 11)},
                    included=False
                ),
            ],
            style={'padding': '20px 20px 20px 20px'}),
            
            html.Div([
                dcc.Checklist(
                    id='checklist',
                    options=[
                        {'label': 'Interpolate', 'value': 'interpolate'},
                        {'label': 'Show Albums in Timeseries', 'value': 'show album'},
                        {'label': 'Boxplot: Weekdays', 'value': 'Boxplot: Weekdays'},
                        {'label': 'Boxplot: Monthly', 'value': 'Boxplot: Monthly'}
                    ],
                    value=[],
                    labelStyle={'display': 'inline-block'}
                )  
            ],
            style={'padding': '20px 20px 20px 20px'}),
            

        ], 
        style={'width':'20%', 'padding': '10px 5px',  'display': 'inline-block', 'vertical-align': 'top'}),
        
        html.Div([
            html.Div([
                dcc.Graph(
                    id='box_plot',
                    hoverData={'points': [{'customdata': 'z'}]}
                )
            ], 
            style={'padding': '0 20'}),

            html.Div([
                dcc.Graph(
                    id='line_graph',
                    hoverData={'points': [{'customdata': 'z'}]}
                )

            ], 
            style={'padding': '0 20'}),
        ], style={'width':'79%', 'display': 'inline-block'})
        
        
    ],
    style={'width': '100%'}),

    html.Div([
        html.Div([
            dcc.Graph(
                id='world_map',
                hoverData={'points': [{'customdata': 'z'}]}
            )
        ], 
        style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            dcc.Graph(
                id='word_cloud',
            )
        ], 
        style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    ], 
    style={'width': '99%', 'display': 'inline-block', 'padding': '0 20'}),    
])

### Tab Management

In [9]:
app.layout = dash_layout
#html.Div([
#    dcc.Tabs(id='tabs-example', value='tab-1', children=[
#        dcc.Tab(label='Settings', value='tab-1'),
#        dcc.Tab(label='Graphs', value='tab-2'),
#    ]),
#    html.Div(id='tabs-example-content')
#])

#@app.callback(dash.dependencies.Output('tabs-example-content', 'children'),
#              [dash.dependencies.Input('tabs-example', 'value')])
#def render_content(tab):
#    if tab == 'tab-1':
#        return settings_layout
#    elif tab == 'tab-2':
#        return dash_layout

## Content

### Datatable

In [10]:
#input_array = np.array([])
#for category in categories:
#    input_array = np.append(input_array, dash.dependencies.Input(category+"_selector", 'value'))
#input_array = input_array.tolist()

#@app.callback(
#    dash.dependencies.Output('data_table', 'data'),
#    input_array)
#def update_data_table(checklist, *kwargs):
#    if not (checklist is None) and not (kwargs is None):
#        for category, function in list(zip(categories, kwargs)):
#            aggr_dict[category] = getattr(np, function)

### Line Plot

In [11]:
@app.callback(
    dash.dependencies.Output('line_graph', 'figure'),
    [dash.dependencies.Input('category_one_selector', 'value'),
     dash.dependencies.Input('category_two_selector', 'value'),
     dash.dependencies.Input('detail_selector', 'value'),
    dash.dependencies.Input('year_selector', 'value'),
    dash.dependencies.Input('averages_selector', 'value'),
    dash.dependencies.Input('checklist', 'value')])
def update_line_graph_figure(category_one, category_two, detail, years, averages, checklist):
    df = data.copy()
    years = np.arange(start=years[0], stop=years[1]+1)
    indices = [np.any(item) for item in list(zip(np.array([df.index.year == year for year in years]).T))]
    df = df[indices].copy()
    df = df.resample(abbreviations[detail]).agg(aggr_dict)

    if 'interpolate' in checklist:
        df.interpolate(method='linear', limit_direction='both', axis=0, inplace=True)
    
    lower_average = averages[0]
    upper_average = averages[1]
    df['%s Moving %s' % (category_one, lower_average)] = df[category_one].rolling(lower_average).mean()
    df['%s Moving %s' % (category_one, upper_average)] = df[category_one].rolling(upper_average).mean()
    df['%s Moving %s' % (category_two, lower_average)] = df[category_two].rolling(lower_average).mean()
    df['%s Moving %s' % (category_two, upper_average)] = df[category_two].rolling(upper_average).mean()

    df = df[[
        category_one, 
        category_two, 
        '%s Moving %s' % (category_one, lower_average),
        '%s Moving %s' % (category_one, upper_average),
        '%s Moving %s' % (category_two, lower_average),
        '%s Moving %s' % (category_two, upper_average)
    ]]
    
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
     
    for category in [category_one, category_two]:
        for column in [category, "%s Moving %s" % (category, lower_average), "%s Moving %s" % (category, upper_average)]:
            fig.add_trace(
                go.Scatter(
                    x=df.index, 
                    y=df[column],
                    name=column, 
                    mode="lines+markers", 
                    marker=dict(
                        opacity=0,
                        size=1,
                        line=dict(
                            width=1,
                        )
                    ),
                    line=dict(
                        width=2,
                        dash="dash" if category_two in column else "solid"
                    ),
                    customdata=np.repeat(column, len(df.index))
                ),
                secondary_y=category_two in column
            )
                    
    # Add figure title
    fig.update_layout(
        title_text="%s & %s Moving Average" % (category_one, category_two),
        hovermode="x unified",
        clickmode='event+select',
        dragmode='drawopenpath',
        newshape_line_color='cyan',
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Time")

    # Set y-axes titles
    fig.update_yaxes(title_text=category_one, secondary_y=False)
    fig.update_yaxes(title_text=category_two, secondary_y=True)
    
    # Add vertical rectangle for every album
    if 'show album' in checklist:
        indices = [np.any(item) for item in list(zip(np.array([df_pictures.index.year == year for year in years]).T))]
        df_pic = df_pictures[indices].copy()
        if len(df_pic.index) > 0:
            for album in df_pic.album.unique():
                dates = df_pic[df_pic.album == album].index
                first_picture_date = dates.min().strftime("%Y-%m-%d")
                last_picture_date = dates.max().strftime("%Y-%m-%d")

                fig.add_vrect(x0=first_picture_date, x1=last_picture_date, y0=df[category_one].min(), y1=df[category_one].max(),
                          #annotation_text=album, annotation_position="top left",
                          fillcolor="green", opacity=0.25, line_width=0, layer="below")

                fig.add_annotation(x=first_picture_date, y=df[category_one].min(),
                    text=album,
                    showarrow=False,
                    yshift=10) 
        
    return fig

### Boxplot

In [12]:
def get_heatmap_plot(xaxis_column_name, yaxis_column_name, detail, year_value, checklist):
    df = data.copy()
    years = np.arange(start=year_value[0], stop=year_value[1]+1)
    indices = [np.any(item) for item in list(zip(np.array([df.index.year == year for year in years]).T))]
    df = df[indices].copy().resample(abbreviations[detail]).agg(aggr_dict)
    
    if 'interpolate' in checklist:
        df.interpolate(method='linear', limit_direction='both', axis=0, inplace=True)

    if 'Week' in detail:
        x = ["%s %s" % (calendar.month_name[1:][item.month-1], item.year) for item in df.index]
        y = [i%4 + 1 for i in range(len(df.index))]
    elif 'Month' in detail:
        x = df.index.year.astype(str)
        y = [calendar.month_name[1:][month-1] for month in df.index.month]
    elif 'Quarter' in detail:
        x = df.index.year.astype(str)
        y = df.index.quarter.astype(str)
    else:
        x = ["%s %s" % (calendar.month_name[1:][item.month-1], item.year) for item in df.index]
        y = df.index.day
        
    return go.Figure(
        data=go.Heatmap(
            z=df[xaxis_column_name],
            x=x,
            y=y,
            colorscale='Viridis'
        )
    )


In [13]:
def get_scatter_plot(xaxis_column_name, yaxis_column_name, detail, year_value, checklist):
    df = data.copy()
    years = np.arange(start=year_value[0], stop=year_value[1]+1)
    indices = [np.any(item) for item in list(zip(np.array([df.index.year == year for year in years]).T))]
    df = df[indices].copy().resample(abbreviations[detail]).agg(aggr_dict)
    df['Date'] = ["%s %s" % (calendar.month_name[1:][item.month-1], item.year) for item in df.index]
    df['Year'] = df.index.year.astype(str)
    df['Size'] = np.repeat(2, len(df.index))
    
    if 'interpolate' in checklist:
        df.interpolate(method='linear', limit_direction='both', axis=0, inplace=True)
        
    return px.scatter(
        df,
        x=xaxis_column_name,
        y=yaxis_column_name,
        color="Year",
        size="Size",
        hover_data=['Date']
    )

In [14]:
def get_densitymap_plot(xaxis_column_name, yaxis_column_name, detail, year_value, checklist):
    df = data.copy()
    years = np.arange(start=year_value[0], stop=year_value[1]+1)
    indices = [np.any(item) for item in list(zip(np.array([df.index.year == year for year in years]).T))]
    df = df[indices].copy().resample(abbreviations[detail]).agg(aggr_dict)
    
    if 'interpolate' in checklist:
        df.interpolate(method='linear', limit_direction='both', axis=0, inplace=True)
    
    fig = px.density_heatmap(
        data_frame=df, 
        x=xaxis_column_name,
        y=yaxis_column_name,
        marginal_x="histogram",
        marginal_y="histogram")
    return fig

In [15]:
def get_box_plot(selected_data, category_one, category_two, detail, years, checklist):
    # Erstelle Boxplot, da dies die letzte Option ist
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    data_selected = not (selected_data is None) and \
            'points' in selected_data.keys() and \
            len(selected_data['points']) > 0 and \
            'customdata' in selected_data['points'][0].keys()
    
    selected_categories = [category_one, category_two]
    years = np.arange(start=years[0], stop=years[1]+1)
    indices = [np.any(group) for group in np.array([data.index.year == year for year in years]).T]
    
    global df
    df = data[indices][selected_categories].copy()
    selected_aggr = {
        category_one: aggr_dict[category_one],
        category_two: aggr_dict[category_two]
    }
        
    df = df.resample(abbreviations[detail]).agg(selected_aggr)
    
    if 'runDuration' in category_one or 'runDuration' in category_two:
        df = df[df['runDuration (min)'] > 0.0]
        
    # Das Boxplot soll nach Wochentagen aufgelistet werden
    if 'Boxplot: Weekdays' in checklist:
        if data_selected:
            points = selected_data['points']
            
            boxes = np.unique([point['customdata'] for point in points])
            
            for box in boxes:
                df_index = pd.to_datetime([point['x'] for point in points if box == point['customdata']])
                days = df_index.weekday.unique().values
                days.sort()
                
                df_data = [point['y'] for point in points if box == point['customdata']]
                global df_temp
                df_temp = pd.DataFrame(data={box: df_data}, index=df_index)
                for day in days:
                    box_data = df_temp[df_temp.index.weekday == day].values.flatten()
                    weekday_name = list(calendar.day_name)[day]
                    
                    fig.add_trace(go.Box(y=box_data, name="{}<br>{}".format(box, weekday_name)),secondary_y=not (category_one in box))
                    
        else:
            for category in selected_categories:
                days = df.index.weekday.unique().values
                days.sort()
                for day in days:
                    box_data = df[df.index.weekday == day][category].values.flatten()
                    weekday_name = list(calendar.day_name)[day]
                    fig.add_trace(go.Box(y=box_data, name="{}<br>{}".format(category, weekday_name)),secondary_y=not (category_one in category))
        
    # Das Boxplot soll nach Monaten aufgelistet werden
    elif 'Boxplot: Monthly' in checklist:
        if data_selected:
            points = selected_data['points']
            
            boxes = np.unique([point['customdata'] for point in points])
            
            for box in boxes:
                df_index = pd.to_datetime([point['x'] for point in points if box == point['customdata']])
                months = df_index.month.unique().values
                months.sort()
                
                df_data = [point['y'] for point in points if box == point['customdata']]
                df_temp = pd.DataFrame(data={box: df_data}, index=df_index)
                for month in months:
                    box_data = df_temp[df_temp.index.month == month].values.flatten()
                    month_name = list(calendar.month_name)[1:][month-1]
                    
                    fig.add_trace(go.Box(y=box_data, name="{}<br>{}".format(box, month_name)),secondary_y=not (category_one in box))
            
        else:
            for category in selected_categories:
                months = df.index.month.unique().values
                months.sort()
                for month in months:
                    box_data = df[df.index.month == month][category].values.flatten()
                    month_name = list(calendar.month_name)[1:][month-1]
                    fig.add_trace(go.Box(y=box_data, name="{}<br>{}".format(category, month_name)),secondary_y=not (category_one in category))
                  
    # Das Boxplot soll nach Kategorien aufgelistet werden
    else:
        if data_selected:
            points = selected_data['points']
            boxes = np.unique([point['customdata'] for point in points])
            for box in boxes:
                df_data = [point['y'] for point in points if box == point['customdata']]
                fig.add_trace(go.Box(y=df_data, name=box), secondary_y=not(category_one in box))
                
        else:
            fig.add_trace(go.Box(y=df[category_one], name=category_one), secondary_y=False)
            fig.add_trace(go.Box(y=df[category_two], name=category_two), secondary_y=True)

    fig.update_yaxes(title_text=category_one, secondary_y=False)
    fig.update_yaxes(title_text=category_two, secondary_y=True)
    return fig

In [16]:
@app.callback(
    dash.dependencies.Output('box_plot', 'figure'),
    [dash.dependencies.Input('plot_type_selector', 'value'),
        dash.dependencies.Input('line_graph', 'selectedData'),
    dash.dependencies.Input('category_one_selector', 'value'),
     dash.dependencies.Input('category_two_selector', 'value'),
     dash.dependencies.Input('detail_selector', 'value'),
    dash.dependencies.Input('year_selector', 'value'),
    dash.dependencies.Input('checklist', 'value')])
def update_boxplot_figure(plot_type, selected_data, category_one, category_two, detail, years, checklist):
    if plot_type == 'Heatmap':
        return get_heatmap_plot(category_one, category_two, detail, years, checklist)
    elif plot_type == 'Scatterplot':
        return get_scatter_plot(category_one, category_two, detail, years, checklist)
    elif plot_type == 'Densitymap':
        return get_densitymap_plot(category_one, category_two, detail, years, checklist)
    elif plot_type == 'Boxplot':
        return get_box_plot(selected_data, category_one, category_two, detail, years, checklist)

### World Map

In [17]:
@app.callback(
    dash.dependencies.Output('world_map', 'figure'),
    [dash.dependencies.Input('year_selector', 'value')])
def update_world_map_figure(years):
    years = np.arange(start=years[0], stop=years[1]+1)
    indices = [np.any(item) for item in list(zip(np.array([df_pictures.index.year == year for year in years]).T))]
    if not np.any(indices):
        df = pd.DataFrame({
            'file':["NaN"],
            'album':["NaN"],
            'datum':["NaN"],
            'Longitude':[0], 
            'Latitude':[0]})
    else:
        df = df_pictures[indices].copy()
        df = pd.DataFrame({
            'file':df['file'],
            'album':df['album'],
            'datum':[date.strftime("%d-%m-%Y") for date in df.index.date],
            'Longitude':df['longitude'], 
            'Latitude':df['latitude']})
    geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
    gdf = GeoDataFrame(df, geometry=geometry)   

    world = gpd.read_file(
        gpd.datasets.get_path('naturalearth_lowres')
    )

    geo_df = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

    if 'album' in gdf.columns:
        return px.scatter_geo(
                    gdf,
                    lat=gdf.geometry.y,
                    lon=gdf.geometry.x,
                    hover_name='file',
                    hover_data=['datum'],
                    color='album'
                )
    else:
        return px.scatter_geo(
                    gdf,
                    lat=gdf.geometry.y,
                    lon=gdf.geometry.x,
                    hover_name='file',
                    hover_data=['datum'],
                )

In [18]:
def get_wordcloud_plot(words, confidence_levels):
    word_count = len(words)
    colors = [px.colors.DEFAULT_PLOTLY_COLORS[random.randrange(1, 10)] for i in range(word_count)]
    scatterplot_data = go.Scatter(
        x=[i%2 for i in range(word_count)],
        y=list(range(word_count)),
        mode='text',
        text=words,
        marker={'opacity': 0.3},
        textfont={
            'size': confidence_levels,
            'color': colors
        }
    )
    layout = go.Layout(
        {
            'xaxis': {
                'showgrid': False, 
                'showticklabels': False, 
                'zeroline': False,
                'range':[-1, 2]
            },
            'yaxis': {
                'showgrid': False, 
                'showticklabels': False, 
                'zeroline': False,
                'range':[-2, word_count]
            }
        }, 
    )
    return go.Figure(data=[scatterplot_data], layout=layout)

def get_words_and_confidence_levels(object, word_confidence_level):
    if 'hovertext' in object:
        filename = object['hovertext']
        words_str = df_pictures[df_pictures['file']==filename]['words'].values[0]
        words_dict = json.loads(words_str)

        image_words = [word for word, confidence in words_dict.items() if float(confidence) > word_confidence_level]
        confidence_levels = [float(confidence)*50 for word, confidence in words_dict.items() if float(confidence) > word_confidence_level]
    else:
        image_words = ["Nothing found."]
        confidence_levels = [1.0]
    
    return image_words, confidence_levels
    
    
def get_words_and_confidence_from_hoverData(hoverData, word_confidence_level):
    for item in hoverData['points']:
        image_words, confidence_levels = get_words_and_confidence_levels(item, word_confidence_level)

        if len(image_words) == 0:
            image_words = ["Nothing found"]
            confidence_levels = [1.0]

        return image_words, confidence_levels

def get_words_and_confidence_from_selectedData(selectedData, word_confidence_level):
    words = {}
    for item in selectedData['points']:
        image_words, confidence_levels = get_words_and_confidence_levels(item, word_confidence_level)

        for word, confidence_level in list(zip(image_words, confidence_levels)):
            words.update({word: confidence_level})
            
    return list(words.keys()), list(words.values())
    
@app.callback(
    dash.dependencies.Output('word_cloud', 'figure'),
    [dash.dependencies.Input('world_map', 'hoverData'),
    dash.dependencies.Input('world_map', 'selectedData'),
    dash.dependencies.Input('word_cloud_count', 'value'),
    dash.dependencies.Input('word_cloud_confidence_level', 'value'),]) 
def update_word_cloud_figure(hoverData, selectedData, word_count, word_confidence_level):
    if selectedData:
        words, confidence_levels = get_words_and_confidence_from_selectedData(selectedData, word_confidence_level)
    elif hoverData:
        words, confidence_levels = get_words_and_confidence_from_hoverData(hoverData, word_confidence_level)
    
    selected_words = {}
    for word, confidence_level in list(zip(words, confidence_levels)):
        selected_words.update({word: confidence_level})
    
    words = selected_words
    if len(words.keys()) > word_count:
        words = dict(list(words.items())[:word_count])
    
    return get_wordcloud_plot(list(words.keys()), list(words.values()))

# Server

In [19]:
# Normale Server-Dashboard Variante, gibt lokale IP für Browser aus:
# app.run_server()

# Um es im Notebook zu sehen:
# app.run_server(mode="inline")

# Um das Dashboard in einem neuen Tab zu öffnen:
app.run_server(mode="jupyterlab", port=1138)

# Circle Plot

In [20]:
import calendar
import datetime

from ipywidgets import Dropdown, SelectMultiple, Checkbox, interact_manual

categories = Dropdown(options=categories, description='Category: ')
diagram_types = Dropdown(options=['Bar', 'Line'], description='Diagram Type: ')
detail = Dropdown(options=time_spans, description='Detail', default='Calendar Day')
years = data.index.year.unique()
year_selector = SelectMultiple(
    options=years,
    value=[],
    rows=len(years),
    description='Jahre',
    disabled=False
)

interp =  Dropdown(options=['Interpolate', 'Fill with 0', 'Keep nan'], description='Missing Values')
overlap = Checkbox(description='Overlap?')
fill = Checkbox(description='Fill?')

@interact_manual(years=year_selector, category=categories, detail=detail, interp=interp, overlap=overlap, fill=fill)
def plot_circle(years, category, detail, interp, overlap, fill):
    diagram_type='Line'
    abbr = abbreviations[detail]
    df = data[category]
    df = df.resample(abbr).agg(aggr_dict[category])
    
    indices = np.array([df.index.year == year for year in years])
    indices = [np.any(index_group) for index_group in indices.T]
    df = df[indices]
    df_length = len(df)
    df_index = df.index
    
    if 'Day' in detail or 'Week' in detail:
        if overlap:
            theta = [360 * (i/(365 if 'Day' in detail else 52)) for year in years for i in range(len(df[df.index.year==year]))]
        else:
            theta = [(i/(df_length + 1)) * 360 for i in range(1, df_length + 1)]
    elif 'Month' in detail:
        theta = [calendar.month_name[1:][np.mod(i, 12)] for i in range(df_length)]
    elif 'Quarter' in detail:
        theta = ["Q%s" % (np.mod(i, 4)+1) for i in range(df_length)]
    else:
        theta = [360 * (i/(len(years)-1)) for i in range(len(years))]

    df = pd.DataFrame({category:df.values, 'Year':df.index.year, 'Theta':theta})
    if 'Fill' in interp:
        df.fillna(0, inplace=True)
    elif 'Inter' in interp:
        df.interpolate(method='linear', inplace=True, limit_direction='both')

    min_value = np.unique(df[category].values)[1]
    range_r = [min_value * 0.9, df[category].max() * 1.1]

    hover_name = None
    if "Year" in detail:
        hover_name = years
    elif "Week" in detail:
        hover_name = ["Week: {}, Year: {}".format(datetime.date(date.year, date.month, date.day).isocalendar()[1], date.year) for date in df_index]
    elif "Calendar Day" in detail or "Business Day" in detail:
        hover_name = ["{}.{}.{}".format(date.day, date.month, date.year) for date in df_index]
    
    fig =  px.line_polar(
        df, 
        r=category, 
        range_r=range_r,
        theta="Theta", 
        color="Year" if "Year" not in detail else None,
        hover_name=hover_name,
        render_mode='auto' if df_length >= 1000 else 'svg',
        color_discrete_sequence=px.colors.sequential.Viridis
    )
    if "Year" in detail:
        num_slices = len(years) - 1
        angular_tickvals = [(i + 1) * 360 / num_slices for i in range(num_slices)]
        fig.update_layout(
            polar_angularaxis_tickvals=angular_tickvals,
            polar = dict(angularaxis = dict(showticklabels=False))
        )
    elif "Week" in detail:
        if overlap:
            fig.update_layout(
                polar_angularaxis_tickvals=np.unique(theta), 
                polar = dict(angularaxis = dict(showticklabels=False))
            )
        else:
            num_slices = len(years) - 1
            angles = list()
            for i in range(len(years)):
                sum_for_year_intervall = 0
                for year in years[:i]:
                    sum_for_year_intervall += len(df[df['Year']==year])
                new_angle = sum_for_year_intervall / len(df) 
                angles.append(360 * new_angle)
            fig.update_layout(polar_angularaxis_tickvals=angles)
    elif "Business Day" in detail or "Calendar Day" in detail:
        num_slices = len(years) - 1
        angles = list()
        for i in range(len(years)):
            sum_for_year_intervall = 0
            for year in years[:i]:
                sum_for_year_intervall += len(df[df['Year']==year])
            new_angle = sum_for_year_intervall / len(df) 
            angles.append(360 * new_angle)
        fig.update_layout(
            polar_angularaxis_tickvals=angles, 
            polar = dict(angularaxis = dict(showticklabels=False))
        )
        
    if fill:
        fig.update_traces(fill='toself')

    return fig

interactive(children=(SelectMultiple(description='Jahre', options=(2010, 2011, 2012, 2013, 2014, 2015, 2016, 2…